In [1]:
import pylab as plt
import pymoc
import xidplus
import numpy as np
%matplotlib inline
from astropy.table import Table, join

/Users/pdh21/anaconda3/envs/xidplus/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
import seaborn as sns

This notebook uses all the raw data from the CIGALE predictions and photoz catalogue, maps, PSF and relevant MOCs to create XID+ prior object and relevant tiling scheme

## Read in MOCs
The selection functions required are the main MOC associated with the masterlist. 

In [3]:
Sel_func=pymoc.MOC()
Sel_func.read('../../dmu4/dmu4_sm_GAMA-12/data/holes_GAMA-12_ukidss_k_O16_20180419_MOC.fits')


## Read in CIGALE predictions catalogue

In [4]:
cigale=Table.read('../../dmu28/dmu28_GAMA-12/data/GAMA-12_Ldust_prediction_results.fits')


In [5]:
cigale['id'].name = 'help_id'


In [6]:
cigale

help_id,bayes.dust.luminosity,bayes.dust.luminosity_err,best.omegacam_u,best.omegacam_g,best.suprime_g,best.decam_g,best.gpc1_g,best.suprime_r,best.gpc1_r,best.omegacam_r,best.decam_r,best.gpc1_i,best.omegacam_i,best.suprime_i,best.gpc1_z,best.vista_z,best.suprime_z,best.decam_z,best.gpc1_y,best.suprime_y,best.vista_y,best.ukidss_y,best.ukidss_j,best.vista_j,best.ukidss_h,best.vista_h,best.vista_ks,best.ukidss_k
,,,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy
bytes28,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
HELP_J112916.236+004455.105,9.652877937681845e+37,1.1766678409173006e+38,0.000980295023013616,0.0015294569259221348,0.0015552267277079985,0.0015603137435057492,0.0015811572677988642,0.003194426484630147,0.0032349965854413334,0.0034445239272319866,0.0036535349776492154,0.005065501198848522,0.005074294272525081,0.005165870926847348,0.005710549140420972,0.005799151537997928,0.005893625255034905,0.006068038481455672,0.00634927742142078,0.006414618774161262,0.0065929279545086,0.0066598767650885874,0.008096326042717792,0.008110674388466112,0.010652489300310508,0.010679241978454539,0.012223806234222284,0.012416388818194771
HELP_J112916.735+004525.955,4.996041519244056e+38,6.0140562531586676e+38,0.002289617379823237,0.004161818705328062,0.00425796740958204,0.004276146711710309,0.004347885113219617,0.009598016200118159,0.009657437961817474,0.010385677257061963,0.011085903901470942,0.01685556324117063,0.017086031391132045,0.017508498251530096,0.022315925077628453,0.02287077476022504,0.023584596917840976,0.02484483030692972,0.027143746691988383,0.02782771221301429,0.029982841008669017,0.030628334494560913,0.04372049906367178,0.04393112752986191,0.06931505703338049,0.06967740395278635,0.09061662122507765,0.09364943803243464
HELP_J112916.868+004538.834,6.743951789320467e+35,2.1173250319725105e+35,0.00797875041146088,0.02980803938006447,0.030587365814790615,0.030750760058357213,0.031623273659674545,0.05743273711118947,0.057787231266229755,0.05988838250592578,0.061753501772432697,0.0808511378305513,0.08167243563101165,0.08331869899532499,0.10187379114129923,0.10419566844502802,0.10698228630537966,0.11402155609826929,0.12581586850318172,0.12929421768016797,0.13785416105492476,0.1401027291304574,0.16353112553667346,0.16385860464563604,0.20705179115301087,0.20852137409255886,0.19551549798399098,0.19135190687006612
HELP_J112916.993+004528.665,1.0694121816308823e+37,1.1878692112187377e+37,0.0009588808608041965,0.002040942316873566,0.002129780788744615,0.002150539571625704,0.0022121266190192946,0.004380283447462689,0.004396330947582648,0.00453091385132659,0.00465871777647253,0.006022178004153719,0.006072130839054361,0.006181126116500576,0.007181560741821394,0.007299813358211741,0.0074405068388268495,0.007810170445369271,0.008488444014190927,0.008622610780257783,0.009027094501232926,0.009142849940643617,0.012212649292989828,0.012232476273615987,0.01452436164549038,0.014569890633646571,0.017942068345588966,0.01780885887519865
HELP_J112917.687+004454.217,8.271422685054857e+37,5.409135851765624e+37,0.0013758533723229232,0.005381981842027911,0.005814869635601862,0.0059033034489654294,0.006142373887441594,0.018297075997784327,0.01837009855440917,0.01949743073052938,0.02052489445477839,0.0319632835878968,0.032430974623685965,0.03345464519735685,0.04379755625187017,0.04475752593662178,0.04613679542662961,0.04983271206440684,0.05561606012836359,0.05802881816076636,0.06285880088413186,0.06402997341269316,0.098059365204896,0.09848184564906226,0.13765475719066567,0.13852205753378716,0.19602762997249945,0.19808723364933625
HELP_J112917.704+004533.215,2.621974984462708e+36,1.3701873131959277e+36,0.0165345729910376,0.08406187751160515,0.0873091714540851,0.08794545854754907,0.09099895711542945,0.19537287285608138,0.1963131

In [7]:
for i in range(0,len(cigale['help_id'])):
    cigale['help_id'][i]=cigale['help_id'][i].strip()
               

## Read in photoz

In [8]:
photoz=Table.read('../../dmu24/dmu24_GAMA-12/data/master_catalogue_gama-12_20171210_photoz_20180410_r_optimised.fits')

In [9]:
photoz

help_id,RA,DEC,id,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,za_hb,chi_r_eazy,chi_r_atlas,chi_r_cosmos,chi_r_stellar,stellar_type
bytes28,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes6
HELP_J115809.625-010119.203,179.54010411484222,-1.022000837333718,415493,0.6959,0.2271,1.1167,0.776,1.7302,1.6579,1.8051,0.023,0.716334851089142,0.002686655,0.0150886325,0.0201354525,0.3918725,m3iii
HELP_J115850.554-004148.083,179.71064084924967,-0.6966898527000007,415494,0.8684,0.4865,1.214,0.752,1.7465,1.6739,1.822,0.043,0.8721001529110377,0.04407825,0.02533725,0.043383625,3.34674,m4iii
HELP_J114948.465+001147.728,177.45193642612378,0.19659122452690234,415495,0.887,0.4999,1.2677,0.724,1.6913,1.595,1.7884,0.07,0.8777164533697708,0.35133575,0.08987975,0.029299375,4.303075,m4v
HELP_J120125.808-000108.647,180.3575328813382,-0.0190684827783537,415496,0.9072,0.583,1.2542,0.75,1.6694,1.6106,1.7305,0.043,0.8946666504926839,0.0040718675,0.0216934375,0.028596175,1.58375425,m4iii
HELP_J120150.572+005703.469,180.4607176743377,0.9509637326107814,415497,1.6554,1.4222,1.822,0.799,-99.0,-99.0,-99.0,-99.0,1.6578975824721525,1.186299,4.031635,2.62545,4.5122975,m7iii
HELP_J120422.567-005913.736,181.09402892998088,-0.9871487696687196,415498,0.8732,0.5548,1.2474,0.702,1.5987,1.4735,1.7224,0.091,0.8889996515380697,0.00577969,0.25491375,0.28173975,0.54081825,m4iii
HELP_J115939.276+003916.795,179.91364829654617,0.6546651721328913,415499,0.7295,0.1838,1.3019,0.615,1.9048,1.595,2.2389,0.183,0.716334851089142,0.00098887325,0.0336144,0.04352285,1.25826925,rk3iii
HELP_J115154.051+010414.748,177.9752121056559,1.070763318504169,415500,0.9787,0.6508,1.3088,0.655,1.6778,1.5335,1.822,0.137,0.9936488758477084,0.199398525,0.020380665,0.048394,4.22107,m5iii
HELP_J120457.625+010258.007,181.2401058180152,1.0494463323665646,415501,0.5947,0.1284,1.2542,0.709,1.8528,1.6819,2.0323,0.087,0.39584643832634897,0.006772955,0.0390784,0.2812145,0.90032575,rk0iii


## Join CIGALE and photoz tables

In [10]:
prior=join(cigale,photoz,keys='help_id')

## Import model/s to find f250_pred


In [11]:
# We have fit a model to the data on other fields to obtained an empirical relationship
# between the ratio (Ldust/f250) and redshift

import pickle

# Function used to fit the data (exponential)
def func(p, x):
    p1, p2 = p
    return p1*np.log(x)+p2

# read model (fitted to all data - XID+)
c_model1a = pickle.load(open('../Ldust_investigation/cosmos_model_usingChi2.pkl', 'rb'))

## Find Ldust/f250 ratio relationship for each model

In [12]:
# ## NEW MODELS - XID+ ##
#######################

# Read model and apply ratio - z function to the dataset
logLflux_ratioC = func(c_model1a.beta, prior['z1_median'])
# Extract the flux prediction from the ratio (Ldust/f250) relationship
Lflux_ratioC = 10**logLflux_ratioC
f_pred = prior['bayes.dust.luminosity'] / Lflux_ratioC


In [13]:
# Different flux cuts
# After an analysis running XID+ on a few tiles with different flux cuts, 
# we decided to use f_pred > 5mJy as our flux cut
print('len total prior list: ', len(prior))
print('len C1 cut > 0.2: ', len(prior[f_pred > 0.2]))
print('len C1 cut > 1: ', len(prior[f_pred > 1]))
print('len C1 cut > 2: ', len(prior[f_pred > 2]))
print('len C1 cut > 3: ', len(prior[f_pred > 3]))
print('len C1 cut > 4: ', len(prior[f_pred > 4]))
print('len C1 cut > 5: ', len(prior[f_pred > 5]))
print('len C1 cut > 7: ', len(prior[f_pred > 7]))
print('len C1 cut > 10: ', len(prior[f_pred > 10]))
print('len C1 cut > 20: ', len(prior[f_pred > 20]))
print('len C1 cut > 50: ', len(prior[f_pred > 50]))
print('len C1 cut > 100: ', len(prior[f_pred > 100]))

len total prior list:  1427798
len C1 cut > 0.2:  1427386
len C1 cut > 1:  1427142
len C1 cut > 2:  1425737
len C1 cut > 3:  1422350
len C1 cut > 4:  1414176
len C1 cut > 5:  1395914
len C1 cut > 7:  1335499
len C1 cut > 10:  1228230
len C1 cut > 20:  933824
len C1 cut > 50:  504308
len C1 cut > 100:  237933


In [14]:
prior_sel = prior[['help_id', 'RA', 'DEC']]
prior_sel['f250_pred'] = f_pred
prior_sel['DEC'].name='Dec'
prior_sel=prior_sel[f_pred>5]

In [15]:
prior_sel

help_id,RA,Dec,f250_pred
bytes28,float64,float64,float64
HELP_J112916.236+004455.105,172.31765184209056,0.7486403137874773,18.208314177694334
HELP_J112916.735+004525.955,172.31972959942718,0.7572096649057047,106.41411903084676
HELP_J112916.868+004538.834,172.32028336523268,0.7607873292267642,23.284148392348794
HELP_J112916.993+004528.665,172.3208057566738,0.7579623882333963,11.849732814662342
HELP_J112917.687+004454.217,172.32369623700535,0.7483935440762677,73.11726389713054
HELP_J112917.704+004533.215,172.32376713092262,0.7592264790677732,309.5360125620595
HELP_J112918.031+004449.789,172.32512732406644,0.7471636659781298,11.375344568712453
HELP_J112918.084+004525.273,172.32534838369415,0.7570203063127026,43.56960031387537
HELP_J112918.563+004439.998,172.32734731211227,0.7444438311510293,41.7513257261498


# Read in Maps

In [21]:

pswfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-12_SPIRE250_v1.0.fits'#SPIRE 250 map
pmwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-12_SPIRE350_v1.0.fits'#SPIRE 350 map
plwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-12_SPIRE500_v1.0.fits'#SPIRE 500 map

#output folder
output_folder='./'

In [22]:
from astropy.io import fits
from astropy import wcs

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist[0].header
im250hdu=hdulist[1].header

im250=hdulist[1].data*1.0E3 #convert to mJy
nim250=hdulist[3].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist[1].header)
pixsize250=3600.0*w_250.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist[0].header
im350hdu=hdulist[1].header

im350=hdulist[1].data*1.0E3 #convert to mJy
nim350=hdulist[3].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist[1].header)
pixsize350=3600.0*w_350.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist[0].header
im500hdu=hdulist[1].header
im500=hdulist[1].data*1.0E3 #convert to mJy
nim500=hdulist[3].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist[1].header)
pixsize500=3600.0*w_500.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()

In [23]:
## Set XID+ prior class

In [24]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu, moc=Sel_func)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(prior_sel['RA'] ,prior_sel['Dec'] ,'GAMA12_Ldust_prediction_results.fits',ID=prior_sel['help_id'] )#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu, moc=Sel_func)
prior350.prior_cat(prior_sel['RA'] ,prior_sel['Dec'] ,'GAMA12_Ldust_prediction_results.fits',ID=prior_sel['help_id'] )
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu, moc=Sel_func)
prior500.prior_cat(prior_sel['RA'] ,prior_sel['Dec'] ,'GAMA12_Ldust_prediction_results.fits',ID=prior_sel['help_id'] )
prior500.prior_bkg(-5.0,5)

In [25]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0,1] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1,1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2,1] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [26]:
import pickle
#from moc, get healpix pixels at a given order
from xidplus import moc_routines
order=9
tiles=moc_routines.get_HEALPix_pixels(order,prior250.sra,prior250.sdec,unique=True)
order_large=6
tiles_large=moc_routines.get_HEALPix_pixels(order_large,prior250.sra,prior250.sdec,unique=True)
print('----- There are '+str(len(tiles))+' tiles required for input catalogue and '+str(len(tiles_large))+' large tiles')
output_folder='./'
outfile=output_folder+'Master_prior.pkl'
xidplus.io.pickle_dump({'priors':[prior250,prior350,prior500],'tiles':tiles,'order':order,'version':xidplus.io.git_version()},outfile)
outfile=output_folder+'Tiles.pkl'
with open(outfile, 'wb') as f:
    pickle.dump({'tiles':tiles,'order':order,'tiles_large':tiles_large,'order_large':order_large,'version':xidplus.io.git_version()},f)
raise SystemExit()

----- There are 5014 tiles required for input catalogue and 100 large tiles
writing total_bytes=1274013253...
writing bytes [0, 1073741824)... done.
writing bytes [1073741824, 1274013253)... done.


SystemExit: 

In [ ]:
prior250.nsrc